In [1]:
import os
import json

from pprint import pprint

from langchain import LLMChain, PromptTemplate
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor, AgentType, initialize_agent
from langchain.llms import vertexai
from langchain.tools.base import BaseTool
from langchain.callbacks.manager import (AsyncCallbackManagerForToolRun, CallbackManagerForToolRun)


from vertexai.preview.language_models import TextGenerationModel
from google.cloud import aiplatform

from pydantic import BaseModel, Field, ConfigDict
from typing import Optional, Type

In [2]:
# trace langchain execution
os.environ["LANGCHAIN_TRACING"] = "false"

# authenticate 
GOOGLE_PROJECT_ID = os.getenv("GOOGLE_PROJECT_ID") # ie, confident-jackle-123456
aiplatform.init(project=GOOGLE_PROJECT_ID, location="us-central1")


## overview
In order to maintain customer privacy and minimize the ability for this agent to leak information, it is not going to have any memory (in the prompt). It is still going to be able to retrieve and relay a user's order, but that will be information retrieved from an api using a tool. 

This saves alot of tokens being fed into the input prompt

In [3]:
from tools import GetOrderIdTool, OrderTool, GetDetailedMenuTool, FindItemIdTool

tools = [GetOrderIdTool(), GetDetailedMenuTool(), FindItemIdTool(), OrderTool()]

In [4]:
agent = initialize_agent(
    tools,
    vertexai.VertexAI(temperature=0),
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [6]:
agent.run("Could I order a 'large coffee'?")



> Entering new AgentExecutor chain...
 Action:
```
{
  "action": "find_item_id_tool",
  "action_input": {
    "name": "large coffee"
  }
}
```

Observation: JHHYV2RFNB3K2DJGR7OM2RCZ
Thought: Action:
```
{
  "action": "order_tool",
  "action_input": {
    "item_id": "JHHYV2RFNB3K2DJGR7OM2RCZ",
    "quantity": "1"
  }
}
```

{'order': {'location_id': 'LETE3KSW0H1RB', 'line_items': [{'catalog_object_id': 'JHHYV2RFNB3K2DJGR7OM2RCZ', 'item_type': 'ITEM', 'quantity': '1'}], 'version': 1}, 'idempotency_key': '2ceb1067c11248f99ffc20cbeff66d1f'}
MKAcxd8OkvMjCdhXY8IBOq7QcSSZY
<ApiResponse {"order": {"id": "MKAcxd8OkvMjCdhXY8IBOq7QcSSZY","location_id": "LETE3KSW0H1RB","line_items": [{"uid": "sRqmuoztwVuSlJuGvxt5cB","catalog_object_id": "4EWWXLTFINIWPH6XSZ7PR7UB","catalog_version": 1694014352283,"quantity": "1","name": "Coffee","variation_name": "Small","base_price_money": {"amount": 200,"currency": "USD"},"gross_sales_money": {"amount": 200,"currency": "USD"},"total_tax_money": {"amount": 0,"cu

'Your order has been placed. Your order ID is MKAcxd8OkvMjCdhXY8IBOq7QcSSZY.'

## Chirp
setup google chirp just for a simple demo